In [ ]:
min(*[1, 2], 0)

In [3]:
names = {
    'newton': 'Метод Ньютона',
    'bisection': 'Метод бисекции',
    'nelder_mead': 'Метод Нелдера-Мида',
    'accelerated_gradient_descent_with_stop': 'Градиентный спуск с ускорением и остановкой',
    'custom_accelerated_gradient_descent': 'Градиентный спуск с ускорением и разворотом',
    'simulated_annealing': 'Алгоритм имитации отжига'
}

In [4]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import os
import random

plt.rcParams['font.family'] = 'Times New Roman'

from typing import Callable
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings(action='ignore', category=UserWarning, module='sklearn')
warnings.catch_warnings()


EPSILON = 1e-3
DATA_DIR = 'CalculationsCache_extra'
MAX_ITERATIONS = 100

CALC_DATA = {
    'dc002_h048': {
        'target_displacement_value': 0.48, 
        'start_stress_value': -350.0,
        'actual_stress': -508.1,
        'N': 1
    },
    'dc006_h080': {
        'target_displacement_value': 0.80, 
        'start_stress_value': -350.0,
        'actual_stress': -327.2,
        'N': 2
    },
    'dc006_h0956': {
        'target_displacement_value': 0.956, 
        'start_stress_value': -350.0,
        'actual_stress': -394.84,
        'N': 3
    },
    'dc008_h084': {
        'target_displacement_value': 0.84, 
        'start_stress_value': -350.0,
        'actual_stress': -264.6,
        'N': 4
    },
    'dc008_h2022': {
        'target_displacement_value': 2.022, 
        'start_stress_value': -350.0,
        'actual_stress': -638.1,
        'N': 5
    }
}

class Experiment:
    data_columns = ['stress', 'displacement', 'time']

    def __init__(self, name, evaluate_approximation: bool = False):
        calc_data = CALC_DATA[name]
        self.name = name
        self.n = calc_data['N']
        self.actual_stress = calc_data['actual_stress']
        self.start_stress_value = calc_data['start_stress_value']
        self.target_displacement_value = calc_data['target_displacement_value']
        self.data = self.read_data()
        self.approximation = self.get_linear_approximation(evaluate_approximation)

    def read_data(self):
        data = pd.read_csv(f'{DATA_DIR}/{self.name}.txt', sep='\t', header=None, names=self.data_columns)
        data['displacement'] = data['displacement'].replace('None', np.nan).astype(float)
        return data

    def get_linear_approximation(self, evaluate: bool = False):

        data = self.data.copy()
        data.dropna(inplace=True)

        X = data[['stress']]
        y = data['displacement']

        model = LinearRegression()
        model.fit(X, y)

        if evaluate:
            y_pred = model.predict(X)
            mae = mean_absolute_error(y, y_pred)
            mse = mean_squared_error(y, y_pred)
            r2 = r2_score(y, y_pred)
            max_abs_error = max(abs(y - y_pred))
            print(f'{self.name}: max_error{max_abs_error:.5f}, mae = {mae:.5f}, mse = {mse:.5f}, r2 = {r2:.5f}')

        def linear_approximation(stress: float | list[float] | np.ndarray | pd.Series | pd.DataFrame) -> float | np.ndarray:
            if isinstance(stress, (list, np.ndarray, pd.Series, pd.DataFrame)):
                result = model.predict(np.array(stress).reshape(-1, 1))
            elif isinstance(stress, (float, int)):
                result = model.predict([[stress]])[0]
            else:
                raise ValueError('Invalid argument type')
            return result

        return linear_approximation
    
    def extrapolate_data(self) -> pd.DataFrame:
        new_data = self.data.copy()

        for stress in range(0, -1001, -10):
            if stress in new_data['stress'].values:
                continue

            closest_stresses = new_data['stress'].values[np.argsort(np.abs(new_data['stress'].values - stress))[:2]]

            if all(np.isnan(new_data[new_data['stress'].isin(closest_stresses)]['displacement'].values)):
                new_displacement = np.nan
            if sum(np.isnan(new_data[new_data['stress'].isin(closest_stresses)]['displacement'].values)) == 1:
                new_displacement = np.nan if random.random() < 0.5 else self.approximation(stress)
            else: 
                new_displacement = self.approximation(stress) # + random.uniform(-0.005, 0.005)

            new_row = pd.DataFrame({
                'stress': [stress], 
                'displacement': [new_displacement], 
                'time': [0]
            })

            new_data = pd.concat([new_data, new_row], ignore_index=True)
        result = new_data.sort_values('stress')
        result = result[result['stress'] >= -1000]
        result.to_csv(f'{DATA_DIR}_extra/{self.name}.txt', sep='\t', index=False, header=False)
        return result

    def plot_approximation(self, extrapolate: bool = False, _ax = None, plot_legend_only: bool = False):
        if extrapolate:
            data = self.extrapolate_data()
        else:
            data = self.data.copy()
        
        if _ax is None:
            _, ax = plt.subplots()
        else:
            ax = _ax

        if plot_legend_only:
            ax.plot([], [], 'o', label='Численное значение', color='tab:red', markersize=1.5)
            ax.plot([], [], color='tab:blue', linewidth=0.7, linestyle='-', label='Метод Ньютона разошёлся')
            ax.plot([], [], '-', color='tab:orange', linewidth=0.7, label='Линейная аппроксимация')

            ax.legend(loc='center')
            ax.axis('off')
            return

        stress_values = pd.DataFrame(np.linspace(min(data['stress']) - 10, max(data['stress']), 100), columns=['stress'])
        displacement_values = self.approximation(stress_values)
        
        ax.plot(stress_values, displacement_values, '-', label='Линейная аппроксимация', color='tab:orange', linewidth=0.7)

        # Add vertical lines for missing values
        done = False
        for stress, displacement in zip(data['stress'], data['displacement']):
            if pd.isna(displacement):
                if not done:
                    label = 'Метод Ньютона разошёлся'
                    done = True
                else:
                    label = None
                ax.plot([stress] * 2, [0, self.approximation(stress)], color='tab:blue', linewidth=0.7, linestyle='-', label=label)
                # ax.plot([stress], [self.approximation(stress)], 'o', color='tab:blue', markersize=1.5, label=label)
        ax.plot(data['stress'], data['displacement'], 'o', label='Численное значение', color='tab:red', markersize=1.5)
        ax.set_xlim(min(data['stress']) - 10, max(data['stress']))
        ax.set_xlabel('Начальное напряжение, МПа')
        ax.set_ylim(0, max(data['displacement'].dropna()) * 1.1)
        ax.set_ylabel('Стрела прогиба, мм')
        ax.legend()
        # ax.set_title(f'Образец №{self.n}', fontdict={'fontsize': 11})
        ax.figure.set_size_inches(5, 3.5)
        plt.tight_layout()
        if _ax is None:
            print('we')
            plt.savefig(f'{self.name}_aprox.png', dpi=300)
            plt.show()

    def apply_method(self, method: Callable, verbose: bool = True, inner_verbose: bool = False, ax = None, plot_legend_only: bool = False):
        result_stress, iterations_data, cals_count = method(self, verbose=inner_verbose)

        if verbose or ax is not None:
            self.create_plot(iterations_data, cals_count, method, ax, plot_legend_only)

        return result_stress, iterations_data, cals_count

    def create_plot(self, iterations_data, cals_count, method, ax=None, plot_legend_only=False):
        GRADIENT = sns.color_palette('rocket', len(iterations_data[0])).as_hex()
        cmap = mcolors.LinearSegmentedColormap.from_list('n', GRADIENT)
        norm = plt.Normalize(0, len(iterations_data[0]) + 1)
        iterations_data = list(zip(*iterations_data))

        if plot_legend_only:
            ax.plot([], [], '--', linewidth=0.7, color='tab:red', label='Начальное значение')
            ax.plot([], [], '--', linewidth=0.7, color='tab:green', label='Экспериментальное значение')
            ax.plot([], [], '-', linewidth=0.7, color='0.8', label='Линейная аппроксимация')
            ax.plot([], [], 'o', markersize=3, color=cmap(norm(0)), label=f'Начальная итерация')
            ax.plot([], [], 'o', markersize=3, color=cmap(norm(9)), label=f'Последняя итерация')
            ax.legend(loc='center')
            ax.axis('off')
            return

        xlims = [min(*iterations_data[0], self.actual_stress), max(*iterations_data[0], self.actual_stress)]
        xlims = [xlims[0] - (xlims[1] - xlims[0]) * 0.05, xlims[1] + (xlims[1] - xlims[0]) * 0.05]
        ylims = [min(*iterations_data[1], self.approximation(self.actual_stress)), max(*iterations_data[1], self.approximation(self.actual_stress))]
        ylims = [ylims[0] - (ylims[1] - ylims[0]) * 0.05, ylims[1] + (ylims[1] - ylims[0]) * 0.05]

        ax.set_xlim(xlims)
        ax.set_ylim(ylims)

        ax.plot(xlims, list(map(self.approximation, xlims)), '-', color='0.8', label='Линейная аппроксимация', linewidth=0.7)
        
        ax.plot(xlims, [iterations_data[1][0]] * 2, '--', color='red', linewidth=0.7)
        ax.plot(xlims, [self.target_displacement_value] * 2, '--', color='green', linewidth=0.7)
        
        scatter = ax.scatter(iterations_data[0], iterations_data[1], marker='o', linewidth=1, c=range(len(iterations_data[0])), s=8, cmap=cmap, norm=norm, label=f'Итерации (1 - {len(iterations_data[0])})')

        for i in range(len(iterations_data[0])):
            ax.plot([iterations_data[0][i]] * 2, [ylims[0], iterations_data[1][i]], '-', color=scatter.cmap(scatter.norm(i)), linewidth=0.5)
            ax.plot([xlims[0], iterations_data[0][i]], [iterations_data[1][i]] * 2, '-', color=scatter.cmap(scatter.norm(i)), linewidth=0.5)
        
        ax.set_title(f'{names[method.__name__]}, {len(iterations_data[0])} ит.', fontdict=dict(fontsize=11))
        # ax.set_title(f'ERR = {abs(iterations_data[1][-1] - self.target_displacement_value):.4f} мм, I = {len(iterations_data[0])}, C = {cals_count})', fontdict=dict(fontsize=11))
        # ax.set_xlabel('Начальное напряжение, МПа')
        # ax.set_ylabel('Стрела прогиба, мм')
        # if not plot_legend_only:
        #     ax.legend()

In [5]:
class Experiments:

    def __init__(self, evaluate_approximation: bool = False):
        self.experiments = {(name := filename.split('.')[0]): Experiment(name, evaluate_approximation) for filename in os.listdir(DATA_DIR) if filename.endswith('.txt')}

    def __getitem__(self, name: str) -> Experiment:
        return self.experiments[name]
    
    def __iter__(self):
        return iter(sorted(self.experiments.values(), key=lambda x: x.n))
    
    def _get_plots_grid(self, num: int = None):
        if num is None:
            num_experiments = len(self.experiments) + 1
        num_cols = 3
        num_rows = (num_experiments + 1) // num_cols
        fig, axs = plt.subplots(num_rows, num_cols, figsize=(4 * num_cols, 2.8 * num_rows))
        if (to_del := num_experiments % num_cols) > 0:
            for i in range(to_del, num_cols):
                axs[-1, i].axis('off')
        return axs.flat

    def plot_approximations(self, one_picture: bool = False):
        if not one_picture:
            for experiment in self:
                fig, ax = plt.subplots(figsize=(6, 3.5))
                experiment.plot_approximation(_ax=ax)
                ax.title
                plt.tight_layout()
                plt.savefig(f'{experiment.name}.png', dpi=300)
                plt.show()
        else:
            axs = self._get_plots_grid()
            plot_legend_only=True
            for experiment, ax in zip([self['dc002_h048']] + list(sorted(self.experiments.values(), key=lambda x: x.n)), axs):
                experiment.plot_approximation(_ax=ax, plot_legend_only=plot_legend_only)
                if plot_legend_only:
                    plot_legend_only = False
            plt.tight_layout()
            plt.savefig('approximations.png', dpi=300)
            plt.show()

    def apply_method(self, method: Callable, verbose: bool = True, inner_verbose: bool = False):

        results = []

        if verbose:
            axs = self._get_plots_grid()
            plot_legend_only=True
            for experiment, ax in zip([self['dc002_h048']] + list(sorted(self.experiments.values(), key=lambda x: x.n)), axs):
                if inner_verbose:
                    print(f'===============================')
                    print(f'Experiment: {experiment.name}')
                result_tuple = experiment.apply_method(method, verbose=verbose, inner_verbose=inner_verbose, ax=ax, plot_legend_only=plot_legend_only)
                if not plot_legend_only:
                    result_stress, iterations_data, cals_count = result_tuple
                    results.append((
                        experiment.name, 
                        result_stress,
                        round(abs(iterations_data[-1][-1] - experiment.target_displacement_value), 5), 
                        len(iterations_data),
                        cals_count
                    ))
                plot_legend_only = False
            plt.tight_layout()
            plt.savefig(f'{method.__name__}_applyings.png', dpi=300)
            plt.show()
        else:    
            for experiment in self:
                if inner_verbose:
                    print(f'===============================')
                    print(f'Experiment: {experiment.name}')
                result_stress, iterations_data, cals_count= experiment.apply_method(method, verbose=verbose, inner_verbose=inner_verbose)
                results.append((
                    experiment.name, 
                    result_stress, 
                    round(abs(iterations_data[-1][-1] - experiment.target_displacement_value), 5), 
                    len(iterations_data),
                    cals_count
                ))

        return pd.DataFrame(results, columns=['experiment', 'result_stress', 'displacement_error', 'iterations', 'calculations'])

In [ ]:
experiments = Experiments(False)
experiments.plot_approximations(True)

In [7]:
def nelder_mead(experiment, verbose=False):
    y_target = experiment.target_displacement_value
    x_0 = experiment.start_stress_value
    f = experiment.approximation
    err = lambda x: abs(f(x) - y_target)
    
    reflection_coeff = 1.0
    expansion_coeff = 2.0
    contraction_coeff = 0.5
    shrinkage_coeff = 0.5
    tolerance = EPSILON

    x_1 = x_0 + 10.0

    iterations_data = []

    step_num = 0
    calcucaltions_number = 2

    while True:
        step_num += 1
        if verbose:
            print(f'Step {step_num}:')
            print(f'x_0 = {round(x_0, 5)}, y_0 = {round(f(x_0), 5)}, err = {round(err(x_0), 5)}')
            print(f'x_1 = {round(x_1, 5)}, y_1 = {round(f(x_1), 5)}, err = {round(err(x_1), 5)}')
            print()

        if err(x_0) > err(x_1):
            x_0, x_1 = x_1, x_0

        iterations_data.append((x_0, f(x_0)))
        
        if err(x_0) <= tolerance:
            return x_0, iterations_data, calcucaltions_number
        
        if step_num >= MAX_ITERATIONS:
            return None, iterations_data, calcucaltions_number

        calcucaltions_number += 1
        x_reflect = x_0 + reflection_coeff * (x_0 - x_1)
        err_reflect = err(x_reflect)

        if err(x_0) <= err_reflect < err(x_1):
            x_1 = x_reflect
        elif err_reflect < err(x_0):
            calcucaltions_number += 1
            x_expand = x_0 + expansion_coeff * (x_reflect - x_0)
            err_expand = err(x_expand)
            if err_expand < err_reflect:
                x_1 = x_expand
            else:
                x_1 = x_reflect
        else:
            calcucaltions_number += 1
            x_contract = x_0 + contraction_coeff * (x_1 - x_0)
            err_contract = err(x_contract)
            if err_contract < err(x_1):
                x_1 = x_contract
            else:
                x_1 = x_0 + shrinkage_coeff * (x_1 - x_0)

In [8]:
def accelerated_gradient_descent_with_stop(experiment, verbose=False):
    y_target = experiment.target_displacement_value
    x = experiment.start_stress_value
    f = experiment.approximation
    err = lambda x: abs(f(x) - y_target)
    
    velocity = 0
    dx = 1e-4
    dt = 25
    tolerance = EPSILON
    iterations_data = []
    prev_acceleration = 0
    calcucaltions_number = 0

    step_num = 0
    while True:
        step_num += 1
        iterations_data.append((x, f(x)))
        if verbose:
            print(f'Step {step_num}:')
            print(f'x = {round(x, 5)}, y = {round(f(x), 5)}, err = {round(err(x), 5)}')
            print()
        
        calcucaltions_number += 1
        if abs(err(x)) <= tolerance:
            return x, iterations_data, calcucaltions_number
        
        if step_num >= MAX_ITERATIONS:
            return None, iterations_data, calcucaltions_number

        calcucaltions_number += 2
        acceleration = (err(x) - err(x + dx)) / dx
        if prev_acceleration * acceleration < 0:
            velocity *= 0
            
        velocity = velocity + acceleration * dt
        x = x + velocity * dt
        prev_acceleration = acceleration

In [9]:
def custom_accelerated_gradient_descent(experiment, verbose=False):
    y_target = experiment.target_displacement_value
    x = experiment.start_stress_value
    f = experiment.approximation
    err = lambda x: abs(f(x) - y_target)
    
    velocity = 0
    dx = 1e-4
    dt = 100
    tolerance = EPSILON
    iterations_data = []
    prev_acceleration = 0
    step_num = 0
    calcucaltions_number = 0

    while True:
        step_num += 1
        if verbose:
            print(f'Step {step_num}:')
            print(f'x = {round(x, 5)}, y = {round(f(x), 5)}, err = {round(err(x), 5)}')
            print()
        
        calcucaltions_number +=1
        iterations_data.append((x, f(x), calcucaltions_number))

        if err(x) <= tolerance:
            return x, iterations_data, calcucaltions_number
        
        if step_num >= MAX_ITERATIONS:
            return None, iterations_data, calcucaltions_number

        calcucaltions_number += 2
        acceleration = (err(x) - err(x + dx)) / dx
        if prev_acceleration * acceleration < 0:
            velocity *= -0.5
            if prev_prev_acceleration * prev_acceleration < 0:
                dt *= 0.5
                velocity *= 0.5

        velocity = velocity + acceleration * dt
        x = x + velocity * dt
        prev_prev_acceleration = prev_acceleration
        prev_acceleration = acceleration

In [10]:
def bisection(experiment, verbose=False):
    y_target = experiment.target_displacement_value
    f = experiment.approximation
    err = lambda x: f(x) - y_target
    
    tolerance = EPSILON
    iterations_data = []

    calcucaltions_number = 0
    step_num = 0

    calcucaltions_number += 2
    a = experiment.start_stress_value - 200
    b = experiment.start_stress_value + 200

    adjustment_step = 200
    max_adjustment_steps = 10
    
    step_num = 0
    while err(a) * err(b) > 0 and step_num < max_adjustment_steps:
        calcucaltions_number += 1
        if step_num % 2 == 0:
            a -= adjustment_step
        else:
            b += adjustment_step

        step_num += 1
        iterations_data.append(((x_mid := (a + b) / 2), f(x_mid)))

    if step_num == max_adjustment_steps:
        print('Max adjustment iterations reached')
        return None, iterations_data, calcucaltions_number

    while True:
        step_num += 1
        x = (a + b) / 2
        iterations_data.append((x, f(x)))
        if verbose:
            print(f'Step {step_num}:')
            print(f'a = {round(a, 5)}, b = {round(b, 5)}')
            print(f'x = {round(x, 5)}, y = {round(f(x), 5)}, err = {round(err(x), 5)}')
            print()

        calcucaltions_number += 1
        if abs(err(x)) <= tolerance:
            return x, iterations_data, calcucaltions_number

        if step_num >= MAX_ITERATIONS:
            return None, iterations_data, calcucaltions_number

        if err(a) * err(x) < 0:
            b = x
        else:
            a = x

In [11]:
def simulated_annealing(experiment, verbose=False):
    y_target = experiment.target_displacement_value
    f = experiment.approximation
    err = lambda x: abs(f(x) - y_target)
    
    initial_temperature = 300
    alpha = 0.7
    max_iterations = MAX_ITERATIONS
    tolerance = EPSILON
    
    x = experiment.start_stress_value
    error = err(x)
    
    temperature = initial_temperature
    iterations_data = []
    
    calcucaltions_number = 1
    step_num = 0
    
    while step_num < max_iterations:
        step_num += 1
        iterations_data.append((x, f(x)))
        
        if verbose:
            print(f'Step {step_num}:')
            print(f'x = {round(x, 5)}, y = {round(f(x), 5)}, err = {round(err(x), 5)}')
            print()
        
        if error < tolerance:
            return x, iterations_data, calcucaltions_number
        
        calcucaltions_number += 1
        cur_x = x + random.uniform(-1, 1) * temperature * 0.1
        cur_error = err(cur_x)
        
        # print(cur_x, cur_error)
        if cur_error < error:
            x = cur_x
            error = cur_error
            temperature *= alpha
        
        
    return None, iterations_data, calcucaltions_number

In [12]:
def newton(experiment, verbose=False):
    y_target = experiment.target_displacement_value
    f = experiment.approximation
    x = experiment.start_stress_value
    err = lambda x: f(x) - y_target
    
    tolerance = EPSILON
    max_iterations = MAX_ITERATIONS
    dx = 10
    
    iterations_data = []
    step_num = 0
    calcucaltions_number = 0
    
    while step_num < max_iterations:
        step_num += 1

        if verbose:
            print(f'Step {step_num}:')
            print(f'x = {round(x, 5)}, y = {round(f(x), 5)}, err = {round(abs(err(x)), 5)}')
            print()
        
        iterations_data.append((x, f(x)))
        
        calcucaltions_number += 1
        if abs(err(x)) < tolerance:
            return x, iterations_data, calcucaltions_number

        calcucaltions_number += 1
        k = (f(x + dx) - f(x)) / dx

        x = x - err(x) / k
    
    return None, iterations_data, calcucaltions_number

In [13]:
# experiments.apply_method(newton, verbose=True, inner_verbose=False)

In [ ]:
experiments.apply_method(bisection, verbose=False, inner_verbose=False)

In [15]:
# experiments.apply_method(simulated_annealing, verbose=True, inner_verbose=False)

In [16]:
# experiments.apply_method(nelder_mead, verbose=True, inner_verbose=False)

In [17]:
# experiments.apply_method(accelerated_gradient_descent_with_stop, verbose=True, inner_verbose=False)

In [18]:
# experiments.apply_method(custom_accelerated_gradient_descent, verbose=True, inner_verbose=False)

In [19]:
names = {
    'newton': 'Метод Ньютона',
    'bisection': 'Метод бисекции',
    'nelder_mead': 'Метод Нелдера-Мида',
    'accelerated_gradient_descent_with_stop': 'Градиентный спуск с ускорением и остановкой',
    'custom_accelerated_gradient_descent': 'Градиентный спуск с ускорением и разворотом',
    'simulated_annealing': 'Алгоритм имитации отжига'
}

In [ ]:
import matplotlib.pyplot as plt

# Generate the data frame
methods = [
    newton,
    bisection,
    nelder_mead,
    accelerated_gradient_descent_with_stop,
    custom_accelerated_gradient_descent, 
    simulated_annealing,
]

start_stress_values = [i for i in range(-850, -50, 5)]
exp = experiments['dc002_h048']
exp.start_stress_value = -460

for method, ax in zip(methods, experiments._get_plots_grid()):
    exp.apply_method(method=method, verbose=True, ax=ax, plot_legend_only=False)
plt.tight_layout()
plt.savefig(f'VISUAL.png', dpi=350)
plt.show()

In [21]:
# data = []
# columns = ['experiment', 'method', 'result_stress', 'iterations', 'calculations', 'start_stress_abs_error', 'start_stress_rel_error']
# plot_data = {}

# for start_stress_value in start_stress_values:
#     for experiment in experiments:
#         experiment.start_stress_value = start_stress_value
#         for method in methods:
#             result_stress, iterations_data, calcs_count = experiment.apply_method(method, verbose=False)
#             data.append([
#                 experiment.name,
#                 method.__name__,
#                 result_stress,
#                 len(iterations_data),
#                 calcs_count,
#                 abs_err := abs(experiment.start_stress_value - experiment.actual_stress),
#                 abs_err / abs(experiment.actual_stress)
#             ])

# df = pd.DataFrame(data, columns=columns)

In [22]:
names = {
    'newton': 'Метод Ньютона',
    'bisection': 'Метод бисекции',
    'nelder_mead': 'Метод Нелдера-Мида',
    'accelerated_gradient_descent_with_stop': 'Градиентный спуск с ускорением и остановкой',
    'custom_accelerated_gradient_descent': 'Градиентный спуск с ускорением и разворотом',
    'simulated_annealing': 'Алгоритм имитации отжига'
}

ordered_names = [
    'nelder_mead',
    'bisection',
    'newton',
    'accelerated_gradient_descent_with_stop',
    'custom_accelerated_gradient_descent',
    'simulated_annealing'
]



In [ ]:
plt.rcParams.update({'font.size': 12})

gr_df = df.groupby('method')

for i, (method, ax) in enumerate(zip(ordered_names, experiments._get_plots_grid())):
    method_df = gr_df.get_group(method)
    data = method_df.groupby('start_stress_abs_error').agg({
        'calculations': 'mean',
        'result_stress': 'first'
    })
    finished_mask = ~data['result_stress'].isna()
    data = data[['calculations']]

    # if method:
    #     print(method)
    #     display(data.sort_values(by='calculations'))

    ax.grid(True, which='both', linewidth=0.3, color='#aaaaaa')
    ax.scatter(data[finished_mask].index, data[finished_mask].values, label='Количество итераций при схождении', s=2)
    
    if data['calculations'].max() != 300 and data['calculations'].max() != 100:
        ax.scatter(data[~finished_mask].index, data[~finished_mask].values, label='Достигнут предел по итерациям', s=2)

    if i in (-1, 3):
        ax.set_ylabel('                                                                Количество вычислений')
    if i in (3,):
        ax.set_xlabel('                                                                                                                                                                                             Отклонение стартового значения напряжения, МПа')
    # ax.set_title(f'{names[method]} – №{i+1}', fontsize=12)
    ax.set_title(f'{names[method]}', fontsize=12)
    # if i in (2, 5):
        # legend = plt.legend(loc="upper right", edgecolor="black")
        # legend
        # legend.get_frame().set_facecolor((0, 0, 1, 0.1))
        # ax.legend().get_frame().set_alpha(None)
# plt.legend()
plt.tight_layout()
plt.savefig(f'methods_comparison.png', dpi=350)
plt.show()

In [ ]:
comparison = {}

for method, method_df in df.groupby('method'):
    method_data = {}

    for exp, exp_df in method_df.groupby('experiment'):
        exp_data = {}
        non_finished_mask = exp_df['result_stress'].isna()
        finished_mask = ~non_finished_mask

        exp_data['mean_fails_ratio'] = non_finished_mask.sum() / len(exp_df)
        exp_data['mean_calculations'] = exp_df[finished_mask]['calculations'].mean()
        exp_data['mean_iterations'] = exp_df[finished_mask]['iterations'].mean()
        exp_data['min_start_stress_abs_error_on_fails'] = exp_df[non_finished_mask]['start_stress_abs_error'].min()
        exp_data['max_start_stress_abs_error'] = exp_df[finished_mask]['start_stress_abs_error'].max()
        # exp_data['min_start_stress_rel_error_on_fails'] = exp_df[non_finished_mask]['start_stress_rel_error'].min()
        # exp_data['max_start_stress_rel_error'] = exp_df[finished_mask]['start_stress_rel_error'].max()

        method_data[exp] = exp_data

    comparison[method] = pd.DataFrame(method_data).T
    comparison[method]['method'] = method

comparison_df = pd.concat(comparison.values())


In [ ]:
aggregated_comparison_df = comparison_df.groupby('method').agg({
    'mean_fails_ratio': 'mean',
    'mean_calculations': 'mean',
    'mean_iterations': 'mean',
    'min_start_stress_abs_error_on_fails': 'min',
    'max_start_stress_abs_error': 'max',
    # 'min_start_stress_rel_error_on_fails': 'min',
    # 'max_start_stress_rel_error': 'max'
}).reset_index()
aggregated_comparison_df.sort_values(
    by=['mean_fails_ratio', 'mean_calculations'],
    ascending=True
)

In [ ]:
# for filename in os.listdir(DATA_DIR):
#     if filename.endswith('.txt'):
#         df = pd.read_csv(f'{DATA_DIR}/{filename}', sep='\t', header=None, names=['stress', 'displacement', 'time'])
#         df['displacement'] = df['displacement'].apply(lambda x: x + random.uniform(-0.000005, 0.000005) if not pd.isna(x) else x)
#         df.to_csv(f'{DATA_DIR}/{filename}', sep='\t', index=False, header=False)